Hey, I'm Jobert Gutierrez and hereafter you'll find the logic and code used to answer the first assignment in the program Machine Learning Operations Zoomcamp offered by Data Talks Club.

# Module 1 Homework: General Overview and foundations
The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Question 1. Downloading the data
We'll use the same [NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

- 16
- 17
- 18
- 19

## Answer:

I downloaded the data using this code:

In [ ]:
# For January 2023:
python -m wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

# For February 2023:
python -m wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet


Then, I read the January file as:

In [4]:
# Calling the libraries used:
import pandas as pd

In [6]:
df = pd.read_parquet(r'C:/Users/jober/OneDrive/Desktop/mlops-hw2025/Homework_week01/yellow_tripdata_2023-01.parquet')

The number of columns is identified using the function:

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Question 2. Computing duration
Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

- 32.59
- 42.59
- 52.59
- 62.59

## Answer:

The duration of the trips are calculated:

In [10]:
# The amount of time for each ride is:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

# Expressed in mutes is:
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

# The stard deviation of the ride's duration is: 
df.duration.std()

42.594351241920904


df = df[(df.duration >= 1) & (df.duration <= 60)]